In [ ]:
# Выгрузите из таблицы track поля milliseconds и bytes. 
# Оба поля должны быть строками. Используйте тип данных для строк нефиксированной длины.
SELECT CAST(milliseconds AS varchar),
       CAST(bytes AS varchar)
FROM track;

In [ ]:
# Выгрузите из таблицы invoice поле total. Оставьте только целую часть числа
SELECT CAST(total AS int4)
FROM invoice;

In [ ]:
# Выгрузите из таблицы staff дни рождения сотрудников. Необходимые значения в виде строк хранит поле birth_date. 
# Сформируйте запрос так, чтобы на экране отобразилась только дата.
SELECT CAST(birth_date AS date)
FROM staff;

In [ ]:
# Выгрузите все значения из таблицы, отфильтровав треки дороже 0.99. Нужные значения хранит столбец unit_price.
SELECT *
FROM invoice_line
WHERE unit_price > 0.99;

In [ ]:
# Из таблицы client выгрузите несколько полей: first_name с именем покупателя, last_name с фамилией и city, 
# в котором указан город проживания. Оставьте записи только о тех покупателях, которые живут в Бразилии (англ. Brazil). 
# Страна проживания хранится в поле country.
SELECT first_name,
       last_name,
       city
FROM client
WHERE country = 'Brazil';

In [ ]:
# Из таблицы invoice выгрузите поля total и customer_id. Значение total должно быть больше или равно 5, 
# а значение customer_id должно равняться 40 или 46.
SELECT total,
       customer_id
FROM invoice
WHERE total >= 5
  AND (customer_id = 40
  OR customer_id = 46);

In [ ]:
# Из таблицы movie выгрузите названия фильмов, стоимость аренды которых не превышает двух долларов, 
# а срок аренды составляет больше шести дней. 
# Выгруженные фильмы не должны относиться к рейтингам PG и PG-13.
SELECT title
FROM movie
WHERE rental_rate <= 2
  AND rental_duration > 6
  AND rating <> 'PG'
  AND rating <> 'PG-13';

In [ ]:
# Выгрузите из таблицы playlist все названия плейлистов из поля name, в которых есть подстрока 'Classic'.
SELECT name
FROM playlist
WHERE name LIKE '%Classic%';

In [ ]:
#  Выгрузите из таблицы movie название фильмов, описание которых заканчивается подстрокой Mexico. 
# Аренда фильмов из итоговой таблицы должна составлять меньше двух долларов либо их рейтинг не должен быть PG-13.
SELECT title
FROM movie
WHERE description LIKE '%Mexico'
  AND (rental_rate < 2
   OR rating <> 'PG-13');

In [ ]:
# Добавьте условие в код задания: выгрузите из таблицы invoice несколько полей, 
# в которых указаны идентификаторы покупателей от 20 до 50 включительно.
SELECT customer_id,
       invoice_date,
       total
FROM invoice
WHERE customer_id BETWEEN 20 AND 50;

In [ ]:
# Отобразите имена, фамилии и страну проживания для покупателей, 
# которые не указали информацию о месте работы, регионе проживания, а также телефон и факс.
SELECT first_name,
       last_name,
       country
FROM client
WHERE company IS NULL
  AND state IS NULL
  AND phone IS NULL
  AND fax IS NULL;

In [ ]:
# Отберите пять самых крупных заказов из таблицы invoice.
SELECT *
FROM invoice
ORDER BY total DESC
LIMIT 5;

In [ ]:
# Отберите пятерых самых активных клиентов в США с 25 мая 2011 по 25 сентября 2011. Дату хранит поле invoice_date
SELECT customer_id,
       COUNT(customer_id)
FROM invoice
WHERE billing_country = 'USA'
  AND CAST(invoice_date AS date) BETWEEN '25-05-2011' AND '25-09-2011'
GROUP BY customer_id
ORDER BY COUNT(customer_id) DESC, customer_id
LIMIT 5;

In [ ]:
# Нужно посмотреть продажи по годам. тсортируйте таблицу по году от большего к меньшему. 
# Отберите только те записи, в которых в поле billing_country указаны страны: США (англ. USA), 
# Великобритания (англ. United Kingdom) и Германия (англ. Germany).
SELECT EXTRACT(YEAR FROM CAST(invoice_date AS date)),
       MIN(total),
       MAX(total),
       SUM(total),
       COUNT(total),
       ROUND(SUM(total) / COUNT(DISTINCT customer_id))
FROM invoice
WHERE billing_country IN ('USA', 'United Kingdom', 'Germany')
GROUP BY EXTRACT(YEAR FROM CAST(invoice_date AS date))
ORDER BY EXTRACT(YEAR FROM CAST(invoice_date AS date)) DESC;

In [ ]:
# Нужно объединить данные двух таблиц: track и invoice_line. 
# Таблица track хранит информацию о музыкальных треках в магазине, названия треков указаны в поле name. 
# Таблица invoice_line содержит данные о купленных треках, их стоимость указана в поле unit_price
SELECT t.name,
       SUM(i.quantity)
FROM track AS t
INNER JOIN invoice_line AS i 
    ON t.track_id = i.track_id 
GROUP BY t.name
LIMIT 20;

In [ ]:
# Нужно дополнить запрос: добавьте поле с идентификатором плейлиста playlist_id
SELECT t.name,
       SUM(i.quantity),
       p.playlist_id
FROM track AS t
INNER JOIN invoice_line AS i 
    ON t.track_id=i.track_id
INNER JOIN playlist_track AS p 
    ON p.track_id = t.track_id
GROUP BY t.name, p.playlist_id
LIMIT 20;

In [ ]:
#  Идентификатор плейлиста теперь указан в итоговой таблице. Но что это за плейлисты — непонятно. 
# Эту информацию можно взять в четвёртой таблице — playlist
SELECT t.name,
       SUM(i.quantity),
       pt.playlist_id,
       p.name
FROM track AS t
INNER JOIN invoice_line AS i 
    ON t.track_id=i.track_id
INNER JOIN playlist_track AS pt 
    ON t.track_id=pt.track_id
INNER JOIN playlist AS p 
    ON p.playlist_id = pt.playlist_id 
GROUP BY t.name, pt.playlist_id , p.name
LIMIT 20;

In [ ]:
# Теперь проанализируйте получившуюся таблицу. Нужно посчитать суммарную стоимость треков для каждого плейлиста
SELECT p.name AS playlist_name,
       SUM(t.unit_price) AS total_revenue
FROM track AS t
INNER JOIN invoice_line AS i 
    ON t.track_id=i.track_id
INNER JOIN playlist_track AS pl 
    ON t.track_id=pl.track_id
INNER JOIN playlist AS p 
    ON pl.playlist_id = p.playlist_id
GROUP BY playlist_name
ORDER BY total_revenue DESC;

In [ ]:
# Отберите названия уникальных категорий фильмов, в которых снималась Эмили Ди (англ. Emily Dee).
SELECT DISTINCT category.name 
FROM film_actor 
INNER JOIN actor 
    ON actor.actor_id = film_actor.actor_id 
INNER JOIN film_category 
    ON film_category.film_id = film_actor.film_id
INNER JOIN category 
    ON category.category_id  = film_category.category_id 
WHERE actor.first_name = 'Emily'
  AND actor.last_name = 'Dee';

In [ ]:
# Выведите названия всех треков, добавив информацию о датах, в которые эти треки покупали. 
SELECT name,
       CAST(invoice.invoice_date AS date)
FROM track 
LEFT JOIN invoice_line 
    ON track.track_id = invoice_line.track_id
LEFT JOIN invoice 
    ON invoice.invoice_id = invoice_line.invoice_id

In [ ]:
# Посчитайте для каждого года число уникальных названий купленных треков
SELECT EXTRACT(YEAR FROM CAST(i.invoice_date AS date)) AS year_of_invoice,
       COUNT(DISTINCT t.name )
FROM track AS t
LEFT JOIN invoice_line AS il 
    ON t.track_id = il.track_id
LEFT JOIN invoice AS i 
    ON il.invoice_id = i.invoice_id
GROUP BY year_of_invoice;

In [ ]:
# У некоторых сотрудников есть менеджеры — их идентификаторы указаны в поле reports_to. 
# Посмотрите внимательно на схему базы: таблица staff отсылает сама к себе. 
# Отобразите таблицу с двумя полями: в первое поле войдут фамилии всех сотрудников, 
# а во второе — фамилии их менеджеров. Назовите поля employee_last_name и manager_last_name
SELECT s2.last_name AS employee_last_name ,
       s1.last_name AS manager_last_name
FROM staff AS s1
RIGHT JOIN staff AS s2
    ON s1.employee_id = s2.reports_to

In [ ]:
# Отобразите названия фильмов, в которых снимались актёры и актрисы, не указанные в базе.
SELECT title
FROM movie 
LEFT JOIN film_actor
    ON movie.film_id = film_actor.film_id
LEFT JOIN actor
    ON film_actor.actor_id = actor.actor_id
WHERE actor.first_name IS NULL
GROUP BY title

In [ ]:
# Отобразите на экране имена исполнителей, для которых в базе данных не нашлось ни одного музыкального альбома.
SELECT name
FROM artist
LEFT JOIN album
    ON artist.artist_id = album.artist_id
GROUP BY name
HAVING COUNT(album.album_id) = 0;